## Purpose

Since the EF calculation requires both the ED image and the ES image. I built a 2-pass model to predict the EF value.
1. In the first pass, I stack a ED and ES image together, and train a encoder-decoder network. The network basically encodes the 2 images down to a feature image. In the decoder stage the network outputs the same ED and ES image again. Hence the feature image becomes the encoding for the ED and ES image.
2. This feature images become input to the resnet backbone. I have a Linear head at the end that will predict the EF value.
3. There are 2 stages of training. First train the Encoder-Decoder network. 
4. Next the pretrained resnet is trained with a regression head. The input to this is the feature image from the encoder. The decoder output is ignored. The output of the resnet is the predicted EF.

In [35]:
import argparse
from datetime import datetime
import os
from tqdm import tqdm

#Pytorch
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn as nn
from torchvision import models, transforms
from torchvision.utils import save_image
#from torch.utils.tensorboard import SummaryWriter
from torch.autograd import Variable
#from torchsummary import summary
from torch.utils.data import Dataset, ConcatDataset, DataLoader, Subset

In [36]:
from pathlib import Path
import tempfile
import PIL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [37]:
torch.cuda.set_device("cuda:0")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [38]:
%env DATA_DIRECTORY =  C:\Workspace\Practice\Python\PyTorch

env: DATA_DIRECTORY=C:\Workspace\Practice\Python\PyTorch


In [39]:
## if environment variable is not set , get a temp directory. 
directory = os.environ.get("DATA_DIRECTORY")
ROOT_DIR = Path(tempfile.mkdtemp()) if directory is None else Path(directory)
print(ROOT_DIR)

C:\Workspace\Practice\Python\PyTorch


In [40]:
def checkPathExists(path):
  if not os.path.exists(path):
    print(f"Cannot access path: {path}")
  else:
    print (f"Path {path} accessible")

In [41]:
CAMUS_ORIGINAL_DATA_DIR = 'CAMUS/original_data/data'
CAMUS_DATA_DIR = 'New_CAMUS_png/CAMUS'

In [42]:
DATA_DIR = ROOT_DIR.joinpath(CAMUS_DATA_DIR)

In [43]:
TRAINING_DATA_DIR = DATA_DIR.joinpath('Training')
TESTING_DATA_DIR = DATA_DIR.joinpath('Testing')
TWO_CHANNEL = '2CH'
FOUR_CHANNEL = '4CH'
PHASE_NAMES = ['ED', 'ES']

In [44]:
### Set the file list as
#[ 
#   ED [(input_file, mask_file), (input_file, mask_file), ....]
#   ES [(input_file, mask_file), (input_file, mask_file), ....]
#]
def data_directories(data_path, class_names, chamber_view):
    num_phases = len(class_names)
    patient_list = [x for x in data_path.iterdir() if x.is_dir()]

    image_files_list = [
        [
            (p, Path(str(p).replace(f"{class_names[i]}", f"{class_names[i]}_gt")))
            for x in patient_list
            for j, p in enumerate(x.glob(f"**/{chamber_view}*{class_names[i]}.png"))
        ]
        for i in range(num_phases)
    ]
    return image_files_list

In [45]:
import pprint

In [46]:
training_2chamber_image_files = data_directories(TRAINING_DATA_DIR, PHASE_NAMES, TWO_CHANNEL)
training_4chamber_image_files = data_directories(TRAINING_DATA_DIR, PHASE_NAMES, FOUR_CHANNEL)
testing_2chamber_image_files = data_directories(TESTING_DATA_DIR, PHASE_NAMES, TWO_CHANNEL)
testing_4chamber_image_files = data_directories(TESTING_DATA_DIR, PHASE_NAMES, FOUR_CHANNEL)

In [47]:
pprint.pprint(training_4chamber_image_files[1])

[(WindowsPath('C:/Workspace/Practice/Python/PyTorch/New_CAMUS_png/CAMUS/Training/patient0001/4CH_ES.png'),
  WindowsPath('C:/Workspace/Practice/Python/PyTorch/New_CAMUS_png/CAMUS/Training/patient0001/4CH_ES_gt.png')),
 (WindowsPath('C:/Workspace/Practice/Python/PyTorch/New_CAMUS_png/CAMUS/Training/patient0002/4CH_ES.png'),
  WindowsPath('C:/Workspace/Practice/Python/PyTorch/New_CAMUS_png/CAMUS/Training/patient0002/4CH_ES_gt.png')),
 (WindowsPath('C:/Workspace/Practice/Python/PyTorch/New_CAMUS_png/CAMUS/Training/patient0003/4CH_ES.png'),
  WindowsPath('C:/Workspace/Practice/Python/PyTorch/New_CAMUS_png/CAMUS/Training/patient0003/4CH_ES_gt.png')),
 (WindowsPath('C:/Workspace/Practice/Python/PyTorch/New_CAMUS_png/CAMUS/Training/patient0004/4CH_ES.png'),
  WindowsPath('C:/Workspace/Practice/Python/PyTorch/New_CAMUS_png/CAMUS/Training/patient0004/4CH_ES_gt.png')),
 (WindowsPath('C:/Workspace/Practice/Python/PyTorch/New_CAMUS_png/CAMUS/Training/patient0005/4CH_ES.png'),
  WindowsPath('C:/Wor

In [48]:
def data_description(image_files_list):
    num_total = len(image_files_list[0])
    image_width, image_height = PIL.Image.open(image_files_list[0][0][0]).size
    print(f"Total Image Count: {num_total}")
    print(f"Image Dimensions: {image_width} x {image_height}")

In [49]:
print(f"Two Chamber Training Data Count")
data_description(training_2chamber_image_files)
print("-------------")
print(f"Four Chamber Training Data Count")
data_description(training_4chamber_image_files)
print("-------------")
print(f"Two Chamber Testing Data Count")
data_description(testing_2chamber_image_files)
print("-------------")
print(f"Four Chamber Testing Data Count")
data_description(testing_4chamber_image_files)
print("-------------")

Two Chamber Training Data Count
Total Image Count: 400
Image Dimensions: 256 x 256
-------------
Four Chamber Training Data Count
Total Image Count: 400
Image Dimensions: 256 x 256
-------------
Two Chamber Testing Data Count
Total Image Count: 50
Image Dimensions: 256 x 256
-------------
Four Chamber Testing Data Count
Total Image Count: 50
Image Dimensions: 256 x 256
-------------


In [50]:
TRAINING_2CH_INFO_DIR = TRAINING_DATA_DIR.joinpath('training_2ch_info')
TRAINING_4CH_INFO_DIR = TRAINING_DATA_DIR.joinpath('training_4ch_info')
TESTING_2CH_INFO_DIR = TESTING_DATA_DIR.joinpath('testing_2ch_info')
TESTING_4CH_INFO_DIR = TESTING_DATA_DIR.joinpath('testing_4ch_info')
checkPathExists(TRAINING_2CH_INFO_DIR)
checkPathExists(TRAINING_4CH_INFO_DIR)
checkPathExists(TESTING_2CH_INFO_DIR)
checkPathExists(TESTING_4CH_INFO_DIR)

Path C:\Workspace\Practice\Python\PyTorch\New_CAMUS_png\CAMUS\Training\training_2ch_info accessible
Path C:\Workspace\Practice\Python\PyTorch\New_CAMUS_png\CAMUS\Training\training_4ch_info accessible
Path C:\Workspace\Practice\Python\PyTorch\New_CAMUS_png\CAMUS\Testing\testing_2ch_info accessible
Path C:\Workspace\Practice\Python\PyTorch\New_CAMUS_png\CAMUS\Testing\testing_4ch_info accessible


In [51]:
def data_info_file(info_dir, chamber_view):
    info_df = pd.DataFrame()
    for file in info_dir.glob(f"**/*.cfg"):
        with open(file) as f: 
            data = f.readlines() 
            data = [x.rstrip('\n') for x in data]
            data = { f"{x.split(': ')[0]}_{chamber_view}": x.split(': ')[1] for x in data}
            data['id'] = str(file.name).split('_')[0]
            info_df = pd.concat([info_df, pd.DataFrame([data])])
    info_df = info_df.reset_index(drop=True)
    info_df = info_df.set_index('id')
    return info_df

In [52]:
training_2chamber_info_df = data_info_file(TRAINING_2CH_INFO_DIR, TWO_CHANNEL)
training_4chamber_info_df = data_info_file(TRAINING_4CH_INFO_DIR, FOUR_CHANNEL)
testing_2chamber_info_df = data_info_file(TESTING_2CH_INFO_DIR, TWO_CHANNEL)
testing_4chamber_info_df = data_info_file(TESTING_4CH_INFO_DIR, FOUR_CHANNEL)

In [53]:
training_2chamber_info_df.LVedv_2CH = training_2chamber_info_df.LVedv_2CH.astype('float32')
training_2chamber_info_df.LVesv_2CH = training_2chamber_info_df.LVedv_2CH.astype('float32')
training_2chamber_info_df.LVef_2CH = training_2chamber_info_df.LVef_2CH.astype('float32')

training_4chamber_info_df.LVedv_4CH = training_4chamber_info_df.LVedv_4CH.astype('float32')
training_4chamber_info_df.LVesv_4CH = training_4chamber_info_df.LVedv_4CH.astype('float32')
training_4chamber_info_df.LVef_4CH = training_4chamber_info_df.LVef_4CH.astype('float32')

testing_2chamber_info_df.LVedv_2CH = testing_2chamber_info_df.LVedv_2CH.astype('float32')
testing_2chamber_info_df.LVesv_2CH = testing_2chamber_info_df.LVedv_2CH.astype('float32')
testing_2chamber_info_df.LVef_2CH = testing_2chamber_info_df.LVef_2CH.astype('float32')

testing_4chamber_info_df.LVedv_4CH = testing_4chamber_info_df.LVedv_4CH.astype('float32')
testing_4chamber_info_df.LVesv_4CH = testing_4chamber_info_df.LVedv_4CH.astype('float32')
testing_4chamber_info_df.LVef_4CH = testing_4chamber_info_df.LVef_4CH.astype('float32')

In [54]:
testing_4chamber_info_df.head()

,ED_4CH,ES_4CH,NbFrame_4CH,Sex_4CH,Age_4CH,ImageQuality_4CH,LVedv_4CH,LVesv_4CH,LVef_4CH
id,,,,,,,,,
patient0001,1,21,21,M,73,Good,165.100006,165.100006,50.400002
patient0002,1,18,18,F,88,Good,78.500000,78.500000,39.500000
patient0003,1,26,26,M,51,Good,120.699997,120.699997,37.799999
patient0004,10,1,10,M,41,Good,182.199997,182.199997,7.900000
patient0005,1,20,20,F,87,Good,73.400002,73.400002,49.299999


## Encoder Decoder

In [55]:
class TubeEncoderDecoder(nn.Module):
    def __init__(self):
        super(TubeEncoderDecoder, self).__init__()
     

        self.encoder = nn.Sequential(
            nn.Conv2d(2, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.ConvTranspose2d(32, 16, 2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 2, stride=2 ),
            nn.ReLU())
        
        self.decoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.ConvTranspose2d(32, 16, 2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 2, 2, stride=2 ),
            nn.ReLU())


    def forward(self, x):
        
        feature_img = self.encoder(x)
        output = self.decoder(feature_img)
                
        return feature_img, output


In [56]:
def prepare_model():
    model = TubeEncoderDecoder()
    model = model.to(device)
    
    return model

In [57]:
## Note that we are only extracting images and not the segmentations.
class StackDataSet(Dataset):
    def __init__(self, data_df, info_df, channel_name):
        self.data_df = data_df
        self.info_df = info_df
        self.transform = transforms.ToTensor()
        self.channel_name = channel_name
        self.mean = info_df[f"LVef_{channel_name}"].mean()
        self.std = info_df[f"LVef_{channel_name}"].std()

    def __len__(self):
        num_total = len(self.data_df[0])
        return num_total

    def __getitem__(self, idx):
        img_loc_ED = self.data_df[0][idx][0]
        img_loc_ES = self.data_df[1][idx][0]

        image_ED = PIL.Image.open(img_loc_ED).convert("L")
        image_ES = PIL.Image.open(img_loc_ES).convert("L")

        #tensor_image = torch.stack((torch.from_numpy(image_ED), torch.from_numpy(image_ES)))
        tensor_image = torch.stack((self.transform(image_ED)[0], self.transform(image_ES)[0]))
        #tensor_image = self.transform(tensor_image)
        return {"image": tensor_image, 
                "lvef_real": self.info_df[f'LVef_{self.channel_name}'][idx],
                "lvef_real_norm": (self.info_df[f'LVef_{self.channel_name}'][idx] - self.mean)/self.std,
                "mean": self.mean,
                "std": self.std
               }

In [58]:
def train_val_dataset(dataset, val_split=0.15):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [59]:
def prepare_data(data_df, info_df, channel_name, split=0.15):
    # Whole dataset
    stacks = StackDataSet(data_df, info_df, channel_name)

    # Split
    datasets = train_val_dataset(stacks, split)

    # Train sub dataset from the whole dataset  
    dataset_train = datasets['train']
    
    # 1 fold to validation
    dataset_val = datasets['val']

    train_size = len(dataset_train)
    val_size = len(dataset_val)

    print("train dataset size =", train_size)
    print("validation dataset size=", val_size)

    dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=2,
                                                  shuffle=False, num_workers= 0)
    
    dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=2,
                                                  shuffle=False, num_workers=0)

    return {"train":dataloader_train, "val":dataloader_val, "dataset_size":{"train": train_size, "val":val_size}}

In [60]:
dataloaders  = prepare_data(training_4chamber_image_files, training_4chamber_info_df, FOUR_CHANNEL)
for i, data in enumerate(dataloaders['train']):
    print(f"{data['image'].shape}, ef={data['lvef_real']}")
    if i == 2:
        break

train dataset size = 340
validation dataset size= 60
torch.Size([2, 2, 256, 256]), ef=tensor([65.3000, 58.2000])
torch.Size([2, 2, 256, 256]), ef=tensor([61.1000, 56.2000])
torch.Size([2, 2, 256, 256]), ef=tensor([58.8000, 54.3000])


In [61]:
def save_model(model, optimizer, epoch, validation_loss, output_path):
   
    check_point_name = f"CAMUS_epoch{epoch}.pt" # get code file name and make a name
    check_point_path = os.path.join(output_path, check_point_name)
    # save torch model
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        # "train_loss": train_loss,
        "val_loss": validation_loss
    }, check_point_path)

In [62]:
def train_model(model, optimizer, criterion, criterion_validation, dataloaders: dict, start_epoch, num_epochs, checkpoint_interval):
    for epoch in tqdm(range(start_epoch, start_epoch + num_epochs)):

        # reset dataloader after some epochs
        for phase in ["train", "val"]:

            if phase == "train":
                model.train()
                dataloader = dataloaders["train"]
            else:
                model.eval()
                dataloader = dataloaders["val"]

            running_loss = 0.0
            running_loss_real = 0.0
            
            for i, sample in tqdm(enumerate(dataloader, 0)):

                # handle input data
                input_img = sample['image']
                input_img = input_img.to(device, torch.float)

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == "train"):

                    feature_img, output_img= model(input_img)
                    #outputs_real = outputs  # * std + mean

                    # Loss
                    loss = criterion(output_img, input_img)
                    loss_real = criterion_validation(output_img , input_img)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                
                # calculate running loss
                running_loss += loss.detach().item() * input_img.size(0)
                running_loss_real+= loss_real.detach().item() * input_img.size(0)

            epoch_loss = running_loss / dataloaders["dataset_size"][phase]
            epoch_loss_real  = running_loss_real / dataloaders["dataset_size"][phase]

            # update tensorboard writer
            #writer.add_scalars("Loss", {phase:epoch_loss}, epoch)
            #writer.add_scalars("Loss_real" , {phase:epoch_loss_real}, epoch)
            
            # update the lr based on the epoch loss
            if phase == "val": 
                # Get current lr
                lr = optimizer.param_groups[0]['lr']
                #print("lr=", lr)
                #writer.add_scalar("LR", lr, epoch)
                # scheduler.step(epoch_loss) 

                # save sample feature grid and image grid
                save_image(input_img[:, 0:1, :, :], str(f"Ip_{epoch}") + ".png", nrow=8, padding=2, normalize=False, value_range=(0,255), scale_each=True, pad_value=0)
                save_image(feature_img[:, 0:1, :, :], str(f"Feat_{epoch}") + ".png", nrow=8, padding=2, normalize=False, value_range=(0,255), scale_each=True, pad_value=0)
                save_image(output_img[:, 0:1, :, :], str(f"Op_{epoch}") + ".png", nrow=8, padding=2, normalize=False, value_range=(0,255), scale_each=True, pad_value=0)
                #writer.add_images("input_one_channel", input_img[:, 0:1, :, :], epoch)

                #writer.add_images("feature_img",feature_img, epoch)

                #writer.add_images("output_one_channel", output_img[:, 0:1, :, :], epoch)
                
            # Print output
            print('Epoch:\t  %d |Phase: \t %s | Loss:\t\t %.4f | Loss-Real:\t %.4f '
                      % (epoch, phase, epoch_loss, epoch_loss_real))
        
        # Save model
        if epoch % checkpoint_interval == 0:
            save_model(model, optimizer, epoch, loss, 'output') # loss = validation loss (because of phase=val at last)

In [63]:
def run_train(data_df, info_df, channel_name):
    model = prepare_model()
    dataloaders = prepare_data(data_df, info_df, channel_name)

    optimizer = optim.Adam(model.parameters(), lr=1e-4 , weight_decay=0)
    # optimizer = optim.SGD(model.parameters(), lr=opt.lr )

    criterion =  nn.MSELoss() # backprop loss calculation
    criterion_validation = nn.L1Loss() # Absolute error for real loss calculations

    # LR shceduler
    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=opt.lr_sch_factor, patience=opt.lr_sch_patience, verbose=True)
    # call main train loop
    train_model(model,optimizer,criterion, criterion_validation, dataloaders, 0, 50, 2)

In [64]:
# Train or retrain or inference

print("Training process is strted..!")
run_train(training_4chamber_image_files, training_4chamber_info_df, FOUR_CHANNEL)

Training process is strted..!
train dataset size = 340
validation dataset size= 60


170it [00:04, 34.26it/s]0:00<?, ?it/s]


Epoch:	  0 |Phase: 	 train | Loss:		 0.0749 | Loss-Real:	 0.1993 


30it [00:00, 40.60it/s]
  2%|▏         | 1/50 [00:05<04:43,  5.78s/it]

Epoch:	  0 |Phase: 	 val | Loss:		 0.0705 | Loss-Real:	 0.1922 


...

Epoch:	  48 |Phase: 	 val | Loss:		 0.0305 | Loss-Real:	 0.0768 


170it [00:04, 34.52it/s]


Epoch:	  49 |Phase: 	 train | Loss:		 0.0312 | Loss-Real:	 0.0778 


30it [00:00, 41.91it/s]
100%|██████████| 50/50 [05:09<00:00,  6.18s/it]

Epoch:	  49 |Phase: 	 val | Loss:		 0.0305 | Loss-Real:	 0.0768 


## Regression using ResNet

In [65]:
def prepare_resnet_model():
    model = models.resnet34(pretrained=True)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 1)
    model = model.to(device)

    return model

In [66]:
def prepare_ae_model(checkpoint_path):
    model = TubeEncoderDecoder()
    checkpoint = torch.load(checkpoint_path)

    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    model = model.to(device)    
    return model

In [67]:
def train_regression_model(model, model_ae, optimizer, criterion, criterion_validation, dataloaders: dict, start_epoch, num_epochs, checkpoint_interval):

    for epoch in tqdm(range(start_epoch, start_epoch + num_epochs)):
        for phase in ["train", "val"]:

            if phase == "train":
                model.train()
                dataloader = dataloaders["train"]
            else:
                model.eval()
                dataloader = dataloaders["val"]

            running_loss = 0.0
            running_loss_real = 0.0
            
            for i, sample in tqdm(enumerate(dataloader, 0)):

                # handle input data
                input_img = sample["image"]
                input_img = input_img.to(device, torch.float)

                # Ground truth data
                gt_real = sample["lvef_real_norm"].reshape((2, 1))
                gt_real = gt_real.to(device, torch.float)

                # get feature image
                feature_img, output_img= model_ae(input_img)
                
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(feature_img)

                    # Loss
                    loss = criterion(outputs, gt_real)
                    loss_real = criterion_validation(outputs , gt_real)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # calculate running loss
                running_loss += loss.detach().item() * input_img.size(0)
                running_loss_real+= loss_real.detach().item() * input_img.size(0)

            epoch_loss = running_loss / dataloaders["dataset_size"][phase]
            epoch_loss_real  = running_loss_real / dataloaders["dataset_size"][phase]

            # update tensorboard writer
            #writer.add_scalars("Loss", {phase:epoch_loss}, epoch)
            #writer.add_scalars("Loss_real" , {phase:epoch_loss_real}, epoch)
            
            # update the lr based on the epoch loss
            if phase == "val": 
                # Get current lr
                lr = optimizer.param_groups[0]['lr']
                print("lr=", lr)
                #writer.add_scalar("LR", lr, epoch)
                # scheduler.step(epoch_loss) 

                # save sample feature grid and image grid
                save_image(feature_img, f"regressionoutput/{epoch}.png", nrow=8, padding=2, normalize=False, value_range=(0,255), scale_each=True, pad_value=0)
                #writer.add_images("input_one_channel", input_img[:, 0:1, :, :], epoch)

                #writer.add_images("feature_img",feature_img, epoch)

                #writer.add_images("output_one_channel", output_img[:, 0:1, :, :], epoch)
                
            # Print output
            print('Epoch:\t  %d |Phase: \t %s | Loss:\t\t %.4f | Loss-Real:\t %.4f '
                      % (epoch, phase, epoch_loss, epoch_loss_real))
        
        # Save model
        if epoch % checkpoint_interval == 0:
            save_model(model, optimizer, epoch, loss, "regressionoutput") # loss = validation loss (because of phase=val at last)

In [68]:
def run_regression_train():
    model = prepare_resnet_model()
    model_ae = prepare_ae_model('output/CAMUS_epoch48.pt')
    dataloaders = prepare_data(training_4chamber_image_files, training_4chamber_info_df, FOUR_CHANNEL, 0.1)

    optimizer = optim.Adam(model.parameters(), lr=1e-3 , weight_decay=0)
    # optimizer = optim.SGD(model.parameters(), lr=opt.lr )

    criterion =  nn.MSELoss() # backprop loss calculation
    criterion_validation = nn.L1Loss() # Absolute error for real loss calculations

    # LR shceduler
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=25, verbose=True)

    # call main train loop
    train_regression_model(model,model_ae, optimizer, criterion, criterion_validation, dataloaders, 0, 50, 1)


In [69]:
# Train or retrain or inference

print("Regression training process is strted..!")
run_regression_train()

Regression training process is strted..!


c:\Users\212364473\.virtualenvs\PyTorch-arlbtak8\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\212364473\.virtualenvs\PyTorch-arlbtak8\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


train dataset size = 360
validation dataset size= 40


180it [00:12, 14.14it/s]0:00<?, ?it/s]


Epoch:	  0 |Phase: 	 train | Loss:		 1.1412 | Loss-Real:	 0.8422 


...

180it [00:13, 13.77it/s]11:30<00:14, 14.83s/it]


Epoch:	  49 |Phase: 	 train | Loss:		 0.1272 | Loss-Real:	 0.2829 


20it [00:00, 22.64it/s]


lr= 0.001
Epoch:	  49 |Phase: 	 val | Loss:		 1.3085 | Loss-Real:	 0.8905 


100%|██████████| 50/50 [11:44<00:00, 14.10s/it]


In [70]:
model_ae = prepare_ae_model('output/CAMUS_epoch48.pt')

In [71]:
model = models.resnet34()
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)
checkpoint = torch.load('regressionoutput/CAMUS_epoch49.pt')
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()
model = model.to(device)    

In [94]:
dataloaders  = prepare_data(training_4chamber_image_files, training_4chamber_info_df, FOUR_CHANNEL, 0.1)
def getfinaldf(split):
    predicted_values = []
    actual_values = []
    for i, data in enumerate(dataloaders[split]):
        #print(f"{data['image'].shape}, ef={data['lvef_real'].reshape((2, 1)).shape}")
        input_img = data['image'].to(device, torch.float)
        feature_img, output_img = model_ae(input_img)
        value = model(feature_img)
        value = value.detach().cpu()
        value1 = torch.mul(value, data['std'][0])
        value1 = torch.add(value1, data['mean'][0]).numpy().flatten()
        #print(f"EF values {value1}")
        #print(f"Real EF values {data['lvef_real']}")
        predicted_values.extend(value1.tolist())
        actual_values.extend(data['lvef_real'].numpy().tolist())
    final_df = pd.DataFrame({'Pred_EF': predicted_values, 'Actual_EF': actual_values, 'Diff_EF': np.abs(np.array(predicted_values)-np.array(actual_values))})
    act_lvef_class = []
    for i in final_df.Actual_EF:
        if i >= 50:
            act_lvef_class.append('Normal')
        elif i > 40:
            act_lvef_class.append('Mild')
        else:
            act_lvef_class.append('Abnormal')
    act_lvef_class = pd.Series(act_lvef_class, name='Actual_HFClass')
    act_lvef_class = act_lvef_class.astype('category')
    act_lvef_class = act_lvef_class.cat.set_categories(["Normal", "Mild", "Abnormal"], ordered=True)
    final_df['Actual_HFClass'] = act_lvef_class
    pred_lvef_class = []
    for i in final_df.Pred_EF:
        if i >= 50:
            pred_lvef_class.append('Normal')
        elif i > 40:
            pred_lvef_class.append('Mild')
        else:
            pred_lvef_class.append('Abnormal')
    pred_lvef_class = pd.Series(pred_lvef_class, name='Pred_HFClass')
    pred_lvef_class = pred_lvef_class.astype('category')
    pred_lvef_class = pred_lvef_class.cat.set_categories(["Normal", "Mild", "Abnormal"], ordered=True)
    final_df['Pred_HFClass'] = pred_lvef_class
    return final_df

train dataset size = 360
validation dataset size= 40


In [95]:
train_output_df = getfinaldf('train')

## Error Analysis

In [96]:

train_output_df


,Pred_EF,Actual_EF,Diff_EF,Actual_HFClass,Pred_HFClass
0,41.314144,28.100000,13.214144,Abnormal,Mild
1,40.726936,26.799999,13.926937,Abnormal,Mild
2,45.455311,39.299999,6.155312,Abnormal,Mild
3,48.623318,42.099998,6.523319,Mild,Mild
4,57.843239,55.400002,2.443237,Normal,Normal
...,...,...,...,...,...
355,80.408737,81.900002,1.491264,Normal,Normal
356,56.215496,50.500000,5.715496,Normal,Normal
357,56.970600,64.300003,7.329403,Normal,Normal
358,37.245880,32.299999,4.945881,Abnormal,Abnormal


In [97]:
from sklearn.metrics import accuracy_score
def Accuracy_ConfusionMatrix(actual, predicted, categories):
    print(f"Accuracy of model: {accuracy_score(actual, predicted)}")
    confusion_matrix = pd.crosstab(actual, predicted, rownames=['Actual'], colnames=['Predicted'])
    print(confusion_matrix)
    print("Sensitivity of model for individual classes")
    class_sum = np.sum(confusion_matrix, axis=1)
    for c,i in enumerate(categories):
        print(f"Class {i} : {confusion_matrix.iloc[c][c]/class_sum[c]}")

In [99]:
print('Confusion Matrix for Training Data')
Accuracy_ConfusionMatrix(train_output_df.Actual_HFClass, 
                         train_output_df.Pred_HFClass,
                         train_output_df.Actual_HFClass.cat.categories)

Confusion Matrix for Training Data
Accuracy of model: 0.6611111111111111
Predicted  Normal  Mild  Abnormal
Actual                           
Normal        212     9         0
Mild           62    21         0
Abnormal       17    34         5
Sensitivity of model for individual classes
Class Normal : 0.9592760180995475
Class Mild : 0.25301204819277107
Class Abnormal : 0.08928571428571429


CAMUS Data is too imbalanced. Hence it works really well on the Normal cases. The other 2 classes do not do well.
Also the Resnet does not seem to learning anything. The loss values is all over the place.